# 广播

广播是在运算期间处理不同形状的张量，这是一种强大的机制，可以让张量的形状在一些情况下自动改变，以便与其他张量进行运算。较小形状的张量会被广播到较大形状的张量，以便使它们的形状兼容。通常其中一个张量的一个维度是1，这种情况下，这个维度会被广播到其他张量的维度上。

## General semantics 基本规则

- Each tensor has at least one dimension. (每个张量至少有一个维度)
- When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist. (当迭代维度大小时，从最后一个维度开始，维度大小必须相等，其中一个为1，或者其中一个不存在)

例如：

In [1]:
import torch

x=torch.empty(5,7,3)  # 创建未初始化指定形状的张量
y=torch.empty(5,7,3)
# same shapes are always broadcastable (i.e. the above rules always hold)

x=torch.empty((0,))
y=torch.empty(2,2)
# x and y are not broadcastable, because x does not have at least 1 dimension

# can line up trailing dimensions
x=torch.empty(5,3,4,1)
y=torch.empty(  3,1,1)
# 1st trailing dimension: both have size 1
# 2nd trailing dimension: y has size 1
# 3rd trailing dimension: x size == y size
# 4th trailing dimension: y dimension doesn't exist

# but:
x=torch.empty(5,2,4,1)
y=torch.empty(  3,1,1)
# x and y are not broadcastable, because in the 3rd trailing dimension 2 != 3

如果两个张量`x` , `y`是“可广播的”，则结果张量大小计算如下：

- 如果`x`和`y`的维数不相等，则在维数较少的张量的维数前面加上 `1`，使它们的长度相等。
- 然后，对于每个维度，结果张量的大小是输入张量的大小的最大值。

例如：

In [2]:
# can line up trailing dimensions to make reading easier
x=torch.empty(5,1,4,1)
y=torch.empty(  3,1,1)
print((x+y).size())

# but not necessary:
x=torch.empty(1)
y=torch.empty(3,1,7)
print((x+y).size())

x=torch.empty(5,2,4,1)
y=torch.empty(3,1,1)
(x+y).size()

torch.Size([5, 3, 4, 1])
torch.Size([3, 1, 7])


RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

## In-place semantics 原地语义

张量的原地操作是指在不更改张量的数据的情况下更改张量的内容。例如，`x.copy_(y)`, `x.t_()`, 将会改变`x`的值。

一个复杂之处是原地操作不允许就地张量因广播而改变形状。

例如：

In [3]:
x=torch.empty(5,3,4,1)
y=torch.empty(3,1,1)
print((x.add_(y)).size())

x=torch.empty(1,3,1)
y=torch.empty(3,1,7)
print((x.add_(y)).size())

torch.Size([5, 3, 4, 1])


RuntimeError: output with shape [1, 3, 1] doesn't match the broadcast shape [3, 3, 7]

## Backwards compatibility 向后兼容性

PyTorch 的早期版本允许在不同形状的张量上执行某些逐点函数，只要每个张量中的元素数量相等。然后通过将每个张量视为一维来执行逐点运算。 PyTorch 现在支持广播，并且“一维”逐点行为被视为已弃用，并且在张量不可广播但具有相同数量元素的情况下将生成 Python 警告。

请注意，在两个张量不具有相同形状但可广播且元素数量相同的情况下，广播的引入可能会导致向后不兼容的更改。例如：

In [4]:
print(torch.ones(4,1).size())
print(torch.randn(4).size())
torch.add(torch.ones(4,1), torch.randn(4))

torch.Size([4, 1])
torch.Size([4])


tensor([[ 2.2196, -0.5307,  1.5837, -0.2263],
        [ 2.2196, -0.5307,  1.5837, -0.2263],
        [ 2.2196, -0.5307,  1.5837, -0.2263],
        [ 2.2196, -0.5307,  1.5837, -0.2263]])

In [5]:
torch.add(torch.ones(4,1), torch.randn( 1))

tensor([[0.2663],
        [0.2663],
        [0.2663],
        [0.2663]])

以前会生成大小为 `torch.Size([4,1])` 的张量，但现在生成大小为 `torch.Size([4,4])` 的张量。为了帮助识别代码中可能存在广播引入的向后不兼容性的情况，您可以将`torch.utils.backcompat.broadcast_warning.enabled`设置为`True` ，这将在这种情况下生成 `python` 警告。

例如：

In [6]:
torch.utils.backcompat.broadcast_warning.enabled=True
torch.add(torch.ones(4,1), torch.ones(4))

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])